In [1]:

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from vgg import *
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import copy
from loguru import logger

logger.debug('All modules imported')

Using TensorFlow backend.
2020-02-04 00:28:39.380 | DEBUG    | __main__:<module>:26 - All modules imported


In [2]:
cur_dir = os.getcwd()
# the data is located in this data_dir
data_dir = os.path.join(cur_dir, 'Dataset')

# initialize the data and labels
logger.debug("[INFO] loading images...")
data = []
labels = []

# grab the image paths and shuffle them
imagePaths = sorted(list(paths.list_images(os.path.join(data_dir, 'Data') )))

# for reproducible results, use a constant seed for shuffling
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    
    # load the image
    image = cv2.imread(imagePath)
    
    # resize it
    image = cv2.resize(image, (32, 32))
    
    # append to the data list
    data.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

logger.debug('[INFO] data loaded complete...')

2020-02-04 00:28:39.392 | DEBUG    | __main__:<module>:6 - [INFO] loading images...
2020-02-04 00:29:04.358 | DEBUG    | __main__:<module>:33 - [INFO] data loaded complete...


In [3]:
data_loaded = copy.deepcopy(data)
labels_loaded = copy.deepcopy(labels)
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data_loaded, dtype="float") / 255.0
labels = np.array(labels_loaded)

# Binarize labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


# Randomly split the data into test and train sets (25% test and 75% train)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)



In [4]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=32, height=32, depth=3,
    classes=len(lb.classes_))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 30
BS = 32

# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

[INFO] training network...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
118/118 [==============================] - 46s 389ms/step - loss: 1.9521 - acc: 0.4118 - val_loss: 1.2778 - val_acc: 0.6284
Epoch 2/30
118/118 [==============================] - 30s 257ms/step - loss: 1.2500 - acc: 0.6030 - val_loss: 0.6936 - val_acc: 0.7655
Epoch 3/30
118/118 [==============================] - 30s 250ms/step - loss: 0.9344 - acc: 0.6946 - val_loss: 0.5504 - val_acc: 0.8090
Epoch 4/30
118/118 [==============================] - 29s 249ms/step - loss: 0.8757 - acc: 0.7171 - val_loss: 0.8059 - val_acc: 0.7559
Epoch 5/30
118/118 [==============================] - 32s 271ms/step - loss: 0.6887 - acc: 0.7648 - val_loss: 3.3573 - val_acc: 0.3106
Epoch 6/30
118/118 [==============================] - 29s 246ms/step - loss: 0.7038 - acc: 0.7685 - val_loss: 0.4819 - val_acc: 0.8368
Epoch 7/30
118/118 [==============================] - 28s 240ms/step

In [6]:
# evaluate the network
print("[INFO] evaluating network...")
print(lb.classes_)
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
['black_dress' 'black_pants' 'black_shoes' 'blue_dress' 'blue_pants'
 'blue_shirt' 'green_shoes' 'red_dress' 'red_shoes' 'white_dress']
              precision    recall  f1-score   support

 black_dress       0.93      0.90      0.91        99
 black_pants       0.93      0.95      0.94       143
 black_shoes       0.88      0.85      0.86       130
  blue_dress       0.90      0.89      0.90       122
  blue_pants       0.87      0.91      0.89       135
  blue_shirt       0.95      0.89      0.92       122
 green_shoes       0.89      0.95      0.92       108
   red_dress       0.97      0.96      0.97       134
   red_shoes       0.94      0.93      0.93       149
 white_dress       0.95      0.97      0.96       120

    accuracy                           0.92      1262
   macro avg       0.92      0.92      0.92      1262
weighted avg       0.92      0.92      0.92      1262



In [8]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('smallvggnet_plot.png')